**Plot Lambs Problem snapshots**

**Daniel Köhn**

**Kiel, 02/03/2018**

**Import necessary packages**

In [ ]:
from __future__ import division
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from matplotlib.colors import LightSource, Normalize
from matplotlib.pyplot import gca
from pylab import rcParams
from matplotlib import rc
import scipy.ndimage.filters
import pickle

**FD grid dimensions **

In [ ]:
DH = 2.0;
NY = 1504;
NX = 1856;
N = NX*NY;

**Wavefield snapshot parameters**

In [ ]:
clip = 3e0         # wavefield clipping
NSNAP1 = 1         # first snapshot
NSNAP2 = 6         # last snapshot
DSNAP = 1          # snapshot increment
TSNAP1 = 2.3e-4    # time of first snapshot
TSNAP2 = 1.38      # time of last snapshot
TSNAPINC = 0.138   # time increment between snapshots

**Define fonts**

In [ ]:
FSize = 22
font = {'color':  'black',
        'weight': 'bold',
        'size': FSize}
mpl.rc('xtick', labelsize=FSize) 
mpl.rc('ytick', labelsize=FSize) 
rcParams['figure.figsize'] = 12, 12

**Read S-wave velocity model and RTM result**

In [ ]:
f = open ('models/modelTest_vp.bin')
data_type = np.dtype ('float32').newbyteorder ('<')
vp = np.fromfile (f, dtype=data_type)
vp = vp.reshape(NX,NY)
vp = np.transpose(vp)
vp = np.flipud(vp)

In [ ]:
def load_snap(offset):
    f1 = open ('snap/waveform_lamb.bin.y')
    data_type1 = np.dtype ('float32').newbyteorder ('<')
    offset_snap = 4*NX*NY*(offset-1);
    snap = np.memmap(f1, dtype=data_type1, mode='c', shape=(NX,NY), offset=offset_snap)
    snap = np.transpose(snap)
    snap = np.flipud(snap)
    return snap

**Define Axis**

In [ ]:
x = np.arange(DH, DH*(NX-1), DH)
y = np.arange(DH, DH*(NY-1), DH)

# convert m -> km
x/=1000.0
y/=1000.0

**Plot $\alpha$-Blending of Vp model (Gray) and snapshots (Seismic)**

In [ ]:
extent = [np.min(x),np.max(x),np.min(y),np.max(y)]

fig = plt.figure(frameon=True)

plt.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
plt.rc('text', usetex=True)

i = NSNAP1
while i <= NSNAP2:
    
    offset = i;
    time = TSNAP1 + (i-1) * TSNAPINC

    snap = load_snap(offset);

    im1 = plt.imshow(vp, cmap=plt.cm.gray, interpolation='nearest', extent=extent, aspect=1)
    plt.hold(True)

    im2 = plt.imshow(snap, cmap=plt.cm.seismic, alpha=.75, interpolation='bicubic',
                     extent=extent, vmin=-clip, vmax=clip, aspect=1)

    # plot source and receiver positions
    plt.plot(1.720,2.299,'r*', markersize=14)
    plt.plot(2.695,2.473,'yv', markersize=10)
    
    plt.xlim(np.min(x),np.max(x))
    plt.ylim(np.min(y),np.max(y))
    
    a = gca()
    a.set_xticklabels(a.get_xticks(), font)
    a.set_yticklabels(a.get_yticks(), font)
    #plt.axis('scaled')
    time_str = str(time)
    #name_title = "Pressure wavefield (time = " + time_str[0:5] + " s)"
    name_title = "Lambs Problem"
    #plt.title(name_title, fontdict=font)
    plt.ylabel('Depth [km]', fontdict=font)
    plt.xlabel('Distance [km]', fontdict=font)
    #plt.gca().invert_yaxis()
    # add annotation
    # plt.text(0.2,3.15,'Computation time: 2.1 s',fontdict=font,color='black',size=30)
    #cbar=plt.colorbar()
    #cbar.set_label('Vp[m/s]', fontdict=font, labelpad=1)
    plt.tight_layout()
    #plt.savefig('Marmousi_RTM.pdf', format='pdf')
    name_snap = "pics/Lamb_snap_" + "%0.*f" %(0,np.fix(offset)) + ".pdf"
    plt.savefig(name_snap, bbox_inches='tight', format='pdf')
    plt.hold(False)
    #plt.show()
    
    i = i + DSNAP;